In [6]:
pip install requests beautifulsoup4

In [ ]:
SportsRadar_APIKEY = "SportRadar_APIKEY"

In [ ]:
# Example to fetch and print all team IDs from the hierarchy endpoint
import requests

url = f"https://api.sportradar.us/nfl/official/trial/v7/en/league/hierarchy.json?api_key={SportsRadar_APIKEY}"
response = requests.get(url)
data = response.json()
teamIDarray = []
for conference in data['conferences']:
    for division in conference['divisions']:
        for team in division['teams']:
            # print(f"Team Name: {team['name']}, Team ID: {team['id']}")
            teamIDarray.append(team['id'])

print(teamIDarray)

['ce92bd47-93d5-4fe9-ada4-0fc681e6caa0', '1f6dcffb-9823-43cd-9ff4-e7a8466749b5', '6680d28d-d4d2-49f6-aace-5292d3ec02c2', '7d4fcc64-9cb5-4d1b-8e75-8a906d1e1576', '768c92aa-75ff-4a43-bcc0-f2798c2e1724', '4809ecb0-abd3-451d-9c4a-92a90b83ca06', '5fee86ae-74ab-4bdd-8416-42a9dd9964f3', '97354895-8c77-4fd4-a860-32e62ea7382a', '82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9', 'f7ddd7fa-0bae-4f90-bc8e-669e4d6cf2de', '82d2d380-3834-4938-835f-aec541e5ece7', 'd26a1ca5-722d-4274-8f97-c92e49c96315', 'ad4ae08f-d808-42d5-a1e6-e9bc4e34d123', 'd5a2eb42-8065-4174-ab79-0a6fa820e35e', 'ebd87119-b331-4469-9ea6-d51fe3ce2f1c', 'cb2f9f1f-ac67-424e-9e72-1475cb0ed398', '4254d319-1bc7-4f81-b4ab-b5e6f3402b69', 'e6aa13a4-0055-48a9-bc41-be28dc106929', 'f14bf5cc-9a82-4a38-bc15-d39f75ed5314', '0d855753-ea21-4953-89f9-0e20aff9eb73', 'f0e724b0-4cbf-495a-be47-013907608da9', 'de760528-1dc0-416a-a978-b510d20692ff', '2eff2a03-54d4-46ba-890e-2bc3925548f3', '3d08af9e-c767-4f88-a7dc-b920c6d2b4a8', '22052ff7-c065-42ee-bc8f-c4691c50e624',

In [ ]:
import pandas as pd
import requests
import time


all_players_data = []

# Function to generate the URL
def generate_url(name):
    formatted_name = name.lower().replace(" ", "-")
    return f"https://www.nfl.com/players/{formatted_name}/stats/"

# Iterate over each team ID
for team_id in teamIDarray:
    success = False
    retries = 5
    while not success and retries > 0:
        # API endpoint to get the roster of a specific team
        url = f"https://api.sportradar.us/nfl/official/trial/v7/en/teams/{team_id}/full_roster.json?api_key={SportsRadar_APIKEY}"

        # Make the API call
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Iterate through the players to collect names and URLs
            for player in data.get('players', []):
                player_name = player['name']
                stats_url = generate_url(player_name)
                all_players_data.append({
                    "Player Name": player_name,
                    "Stats URL": stats_url
                })

            success = True  # Mark as successful
            time.sleep(1)  # Add a small delay to prevent rapid-fire requests
        elif response.status_code == 429:
            print(f"Rate limit hit for team {team_id}. Retrying in 30 seconds...")
            time.sleep(30)  # Wait 30 seconds before retrying
            retries -= 1  # Decrease the retry counter
        else:
            print(f"Failed to retrieve data for team {team_id}: {response.status_code}")
            success = True  # Exit the loop for this team even if it failed

# Convert the accumulated data to a DataFrame
df = pd.DataFrame(all_players_data)

# Save the combined data to a CSV file
df.to_csv('nfl_players_stats_urls.csv', index=False)

# Optionally, display the DataFrame
print(df)


Rate limit hit for team 1f6dcffb-9823-43cd-9ff4-e7a8466749b5. Retrying in 30 seconds...
               Player Name                                          Stats URL
0            Keidron Smith   https://www.nfl.com/players/keidron-smith/stats/
1           Kristian Welch  https://www.nfl.com/players/kristian-welch/stats/
2            Garett Bolles   https://www.nfl.com/players/garett-bolles/stats/
3           Damarri Mathis  https://www.nfl.com/players/damarri-mathis/stats/
4            Jordan Miller   https://www.nfl.com/players/jordan-miller/stats/
...                    ...                                                ...
2449       Jaylin Williams  https://www.nfl.com/players/jaylin-williams/st...
2450          Walter Rouse    https://www.nfl.com/players/walter-rouse/stats/
2451         Ivan Pace Jr.   https://www.nfl.com/players/ivan-pace-jr./stats/
2452  Levi Drake Rodriguez  https://www.nfl.com/players/levi-drake-rodrigu...
2453       Jordan Kunaszyk  https://www.nfl.com/player

In [70]:
import pandas as pd
df = pd.read_csv('nfl_players_stats_urls.csv')
df.head()

,Player Name,Stats URL
0,Keidron Smith,https://www.nfl.com/players/keidron-smith/stats/
1,Kristian Welch,https://www.nfl.com/players/kristian-welch/stats/
2,Garett Bolles,https://www.nfl.com/players/garett-bolles/stats/
3,Damarri Mathis,https://www.nfl.com/players/damarri-mathis/stats/
4,Jordan Miller,https://www.nfl.com/players/jordan-miller/stats/


In [71]:
player_urls = df['Stats URL'].tolist()
print(player_urls)

['https://www.nfl.com/players/keidron-smith/stats/', 'https://www.nfl.com/players/kristian-welch/stats/', 'https://www.nfl.com/players/garett-bolles/stats/', 'https://www.nfl.com/players/damarri-mathis/stats/', 'https://www.nfl.com/players/jordan-miller/stats/', 'https://www.nfl.com/players/p.j.-locke/stats/', 'https://www.nfl.com/players/quinn-bailey/stats/', 'https://www.nfl.com/players/zach-wilson/stats/', 'https://www.nfl.com/players/nick-gargiulo/stats/', 'https://www.nfl.com/players/devaughn-vele/stats/', 'https://www.nfl.com/players/matt-henningsen/stats/', 'https://www.nfl.com/players/audric-estime/stats/', 'https://www.nfl.com/players/levi-wallace/stats/', 'https://www.nfl.com/players/ronnie-perkins/stats/', 'https://www.nfl.com/players/mitchell-fraboni/stats/', 'https://www.nfl.com/players/calvin-throckmorton/stats/', 'https://www.nfl.com/players/adam-trautman/stats/', 'https://www.nfl.com/players/donald-parham/stats/', 'https://www.nfl.com/players/luke-wattenberg/stats/', 'h

In [24]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Setup Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run Chrome in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the Chrome driver once before the loop
driver = webdriver.Chrome(options=options)  # Use the correct path to chromedriver

# Initialize an empty list to store player stats
player_stats_dict = []

# Iterate over each player URL
for url in player_urls:
    player_stats = {}
    try:
        # Open the NFL player stats page
        driver.get(url)

        # Optionally, wait for the page to load completely
        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'nfl-c-player-header__title')))

        # Get the page source after the page has loaded
        page_source = driver.page_source

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract player name
        try:
            player_name = soup.find('h1', class_='nfl-c-player-header__title').text.strip()
        except AttributeError:
            player_name = 'Unknown'

        # Extract player position
        try:
            position = soup.find('span', class_='nfl-c-player-header__position').text.strip()
        except AttributeError:
            position = 'Unknown'

        # Extract player team
        try:
            team = soup.find('a', class_='nfl-o-cta--link').text.strip()
        except AttributeError:
            team = 'Unknown'

        # Initialize career stats as None
        career_stats = None

        try:
            # Locate the table with career stats
            table = soup.find('table', attrs={'summary': 'Career Stats'})  # Adjust this if needed to match the site's HTML

            if table:
                headers = [header.text.strip() for header in table.find_all('th')]
                career_stats = []
                for row in table.find_all('tr')[1:]:  # Skip the header row
                    columns = row.find_all('td')
                    if columns:
                        career_stats.append({headers[i]: columns[i].text.strip() for i in range(len(columns))})
        except Exception as e:
            print(f"An error occurred while processing the table: {e}")

        player_stats = {
            'Player': player_name,
            'Position': position,
            'Career Stats': career_stats,
            'Team': team
        }
        player_stats_dict.append(player_stats)

    except Exception as e:
        print(f"An error occurred while processing the URL {url}: {e}")
    print(player_stats)

# Quit the WebDriver after the loop
driver.quit()

{'Player': 'Keidron Smith', 'Position': 'DB', 'Career Stats': [], 'Team': 'Denver Broncos'}
{'Player': 'Kristian Welch', 'Position': 'LB', 'Career Stats': [{'SEASON': '2020', 'TEAM': 'Baltimore Ravens', 'G': '10', 'GS': '0', 'TKL': '3', 'AST': '2', 'COMBINED': '5', 'SOLO': '2', 'SCK': '0', 'SFTY': '0', 'PDEF': '0', 'INT': '', 'YDS': '', 'AVG': '', 'LNG': '', 'TDS': '', 'FF': '0', 'OPP FR': ''}, {'SEASON': '2021', 'TEAM': 'Baltimore Ravens', 'G': '16', 'GS': '0', 'TKL': '3', 'AST': '10', 'COMBINED': '13', 'SOLO': '3', 'SCK': '0', 'SFTY': '0', 'PDEF': '0', 'INT': '', 'YDS': '', 'AVG': '', 'LNG': '', 'TDS': '', 'FF': '0', 'OPP FR': ''}, {'SEASON': '2022', 'TEAM': 'Baltimore Ravens', 'G': '17', 'GS': '0', 'TKL': '3', 'AST': '0', 'COMBINED': '3', 'SOLO': '2', 'SCK': '0', 'SFTY': '0', 'PDEF': '0', 'INT': '', 'YDS': '', 'AVG': '', 'LNG': '', 'TDS': '', 'FF': '0', 'OPP FR': ''}, {'SEASON': '2023', 'TEAM': 'Green Bay Packers', 'G': '14', 'GS': '0', 'TKL': '4', 'AST': '2', 'COMBINED': '6', 'SOLO

In [67]:
import pandas as pd

# Assuming the DataFrame 'df' is already created and combined as in the previous example
FinalDF = pd.DataFrame(player_stats_dict)
# Function to extract 2023 stats as a dictionary
def extract_2023_stats(career_stats):
    if isinstance(career_stats, list):
        for stats in career_stats:
            if stats.get('SEASON') == '2023':
                return stats
    return {}

# Apply the function to the 'Career Stats' column and convert it to a DataFrame
df_2023_stats = FinalDF['Career Stats'].apply(extract_2023_stats).apply(pd.Series)

# Combine the original DataFrame with the new DataFrame containing the 2023 stats
df_combined = pd.concat([FinalDF.drop(columns=['Career Stats']), df_2023_stats], axis=1)

# Rename 'Team' to 'Current Team'
df_combined.rename(columns={'Team': 'Current Team'}, inplace=True)

# Remove any unnecessary columns (if '2023 Stats' column exists after processing)
if '2023 Stats' in df_combined.columns:
    df_combined.drop(columns=['2023 Stats'], inplace=True)

# Display the final DataFrame
df_combined.head()

,Player,Position,Current Team,SEASON,TEAM,G,GS,TKL,AST,COMBINED,...,SCK,SFTY,PDEF,INT,YDS,AVG,LNG,TDS,FF,OPP FR
0,Keidron Smith,DB,Denver Broncos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Kristian Welch,LB,Denver Broncos,2023,Green Bay Packers,14,0,4,2,6,...,0,0,0,,,,,,0,
2,Garett Bolles,OT,Denver Broncos,2023,Denver Broncos,17,17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Damarri Mathis,CB,Unknown,2023,Denver Broncos,17,6,26,9,35,...,0,0,1,,,,,,0,
4,Jordan Miller,DB,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
